In [9]:

!pip install weaviate-client

In [10]:
!pip install  langchain

In [11]:
!pip install tiktoken

In [12]:
!pip install pypdf

In [13]:
!pip install rapidocr-onnxruntime

In [14]:
!pip install -U langchain-community

In [50]:

WEAVIATE_CLUSTER="https://xddwtnret9al7zpcayszbg.c0.us-west3.gcp.weaviate.cloud"   # it is the url
WEAVIATE_API_KEY="wwcIJzMWfcF66K2EtbT4nhuMjwIrkLU5Fzwv"                   # it is API key

In [51]:
from langchain.vectorstores import Weaviate
import weaviate   # we are using vector data base called as WEAVIATE

In [52]:

WEAVIATE_URL = WEAVIATE_CLUSTER
WEAVIATE_API_KEY = WEAVIATE_API_KEY

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)                    # here we are passing both to create client
)

In [53]:

# to handle , fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [30]:
!pip install sentence-transformers

In [34]:
# specify embedding model (using huggingface sentence transformer)
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
#model_kwargs = {"device": "cuda"}

In [35]:

embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  #model_kwargs=model_kwargs
)

# we will provide the PDF to perfrom task wrt PDF and Load that PDF , spiit the data into Chunks , perfrom embeddig

In [36]:

from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/elctric_charges_book_RAG.pdf", extract_images=True)
pages = loader.load()

In [37]:

pages

[Document(metadata={'source': '/content/elctric_charges_book_RAG.pdf', 'page': 0}, page_content='Chapter One\nELECTRIC CHARGES\nAND FIELDS\n1.1  I NTRODUCTION\nAll of us have the experience of seeing a spark or hearing a crackle when\nwe take of f our synthetic clothes or sweater , particularly in dry weather .\nHave you ever tried to find any explanation for this phenomenon? Another\ncommon example of electric discharge is the lightning that we see in the\nsky during thunderstor ms. W e also experience a sensation of an electric\nshock either while opening the door of a car or holding the iron bar of a\nbus after sliding from our seat. The reason for these experiences is\ndischarge of electric charges through our body, which were accumulated\ndue to rubbing of insulating sur faces. Y ou might have also hear d that\nthis is due to generation of static electricity. This is precisely the topic we\nare going to discuss in this and the next chapter . Static means anything\nthat does not mo

# Split text into chunks

In [45]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

docs = text_splitter.split_documents(pages)   # provide data

In [46]:

docs

[Document(metadata={'source': '/content/elctric_charges_book_RAG.pdf', 'page': 0}, page_content='Chapter One\nELECTRIC CHARGES\nAND FIELDS\n1.1  I NTRODUCTION\nAll of us have the experience of seeing a spark or hearing a crackle when\nwe take of f our synthetic clothes or sweater , particularly in dry weather .\nHave you ever tried to find any explanation for this phenomenon? Another\ncommon example of electric discharge is the lightning that we see in the\nsky during thunderstor ms. W e also experience a sensation of an electric\nshock either while opening the door of a car or holding the iron bar of a\nbus after sliding from our seat. The reason for these experiences is\ndischarge of electric charges through our body, which were accumulated\ndue to rubbing of insulating sur faces. Y ou might have also hear d that\nthis is due to generation of static electricity. This is precisely the topic we\nare going to discuss in this and the next chapter . Static means anything\nthat does not mo

In [47]:
docs[3]

Document(metadata={'source': '/content/elctric_charges_book_RAG.pdf', 'page': 1}, page_content='an entry which is called the electric char ge. We say that the bodies like\nglass or plastic rods, silk, fur and pith balls are electrified. They acquire\nan electric charge on rubbing. T here are two kinds of electrification and\nwe find  that (i) like charges repel  and (ii) unlike charges  attract  each\nother . The pr operty which dif ferentiates the two kinds of char ges is called\nthe polarity  of charge.\nWhen a glass rod is rubbed with silk, the rod acquires one kind of\ncharge and the silk acquires the second kind of charge. This is true for\nany pair of objects that are rubbed to be electrified. Now if the electrified\nglass rod is brought in contact with silk, with which it was rubbed, they\nno longer attract each other . They also do not attract or r epel other light\nobjects as they did on being electrified.\nThus, the charges acquired after rubbing are lost when the charged\nbo

In [49]:
len(docs)

124

In [ ]:
#   docs= spltted data into chunk
#  embeddings =  we use hugginface embedding
#  client = vector db called Weaviate

In [54]:
vector_db = Weaviate.from_documents(
    docs, embeddings, client=client, by_text=False
)


Now we use similarity search method to find user given Question

In [ ]:
# what we called unit of charge ?
# unit of charge is called what ?

In [56]:
print(vector_db.similarity_search("what we called unit of charge ?", k=3))

[Document(metadata={'page': 4, 'source': '/content/elctric_charges_book_RAG.pdf'}, page_content='was first suggested by the experimental laws of electrolysis discovered by\nEnglish experimentalist Faraday. It was experimentally demonstrated by\nMillikan in 1912.\nIn the International System (SI) of Units, a unit of charge is called a\ncoulomb  and is denoted by the symbol C. A coulomb is defined in terms\nthe unit of the electric current which you are going to learn in a\nsubsequent chapter . In ter ms of this definition, one coulomb is the char ge\nflowing through a wire in 1 s if the current is 1 A (ampere), (see Chapter 1\nof Class XI, Physics Textbook , Part I). In this system, the value of the\nbasic unit of charge is\ne = 1.602192 × 10–19 C\nThus, there are about 6 × 1018 electrons in a charge of  –1C. In\nelectrostatics, charges of this large magnitude are seldom  encountered\nand hence we use smaller units 1 mC (micro coulomb) = 10–6 C or 1 mC\n(milli coulomb) = 10–3 C.\nIf the

In [62]:
print(vector_db.similarity_search("what we called unit of charge ?", k=3)[0].page_content)

was first suggested by the experimental laws of electrolysis discovered by
English experimentalist Faraday. It was experimentally demonstrated by
Millikan in 1912.
In the International System (SI) of Units, a unit of charge is called a
coulomb  and is denoted by the symbol C. A coulomb is defined in terms
the unit of the electric current which you are going to learn in a
subsequent chapter . In ter ms of this definition, one coulomb is the char ge
flowing through a wire in 1 s if the current is 1 A (ampere), (see Chapter 1
of Class XI, Physics Textbook , Part I). In this system, the value of the
basic unit of charge is
e = 1.602192 × 10–19 C
Thus, there are about 6 × 1018 electrons in a charge of  –1C. In
electrostatics, charges of this large magnitude are seldom  encountered
and hence we use smaller units 1 mC (micro coulomb) = 10–6 C or 1 mC
(milli coulomb) = 10–3 C.
If the protons and electrons are the only basic charges in the


In [63]:

print(vector_db.similarity_search("what we called unit of charge ?", k=3)[1].page_content)

Electric Charges
and Fields
5in a process. Sometimes nature creates charged particles: a neutron turns
into a proton and an electron. The proton and electron thus create d have
equal and opposite charges and the total charge is zero before and after
the creation.
1.4.3  Quantisation of charge
Experimentally it is established that all free charges are integral multiples
of a basic unit of charge denoted by e. Thus charge q on a body is always
given by
q = ne
wher e n is any integer , positive or negative. This basic unit of char ge is
the charge that an electron or proton carries. By convention, the charge
on an electron is taken to be negative; therefore charge on an electron is
written as –e and that on a proton as +e.
The fact that electric charge is always an integral multiple of e  is termed
as quanti sation of charge . There are a large number of situations in physics
where certain physical quantities are quanti sed. The quanti sation of charge


In [64]:
print(vector_db.similarity_search("what we called unit of charge ?", k=3)[2].page_content)

charge distribution. The linear charge density  l of a wire is defined by
Q
lλΔ=Δ(1.24)
where Dl is a small line element of wire on the macroscopic scale that,
however , includes a lar ge number of micr oscopic char ged constituents,
and DQ is the charge contained in that line element. The units for l are
C/m. The volume charge density  (sometimes simply called charge density)
is defined in a similar manner:
Q
VρΔ=Δ(1.25)
where DQ is the charge included in the macroscopically small volume
element DV that includes a large number of microscopic charged
constituents. The units for r are C/m3.
The notion of continuous charge distribution is similar to that we
adopt for continuous mass distribution in mechanics. When we refer toFIGURE  1.21
Definition of linear ,
surface and volume
charge densities.
In each case, the
element ( Dl, DS, DV)
chosen is small on
the macroscopic
scale but contains
a very large number
of microscopic
constituents.


Now we gonna define a prompt

In [88]:

from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [89]:

prompt=ChatPromptTemplate.from_template(template)

In [90]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"))])

In [91]:
from langchain import HuggingFaceHub


In [92]:

from google.colab import userdata
huggingfacehub_api_token=userdata.get('HuGGINGFACE_TOKEN')

# mistralai/Mistral-7B-v0.1

In [93]:
model = HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":1, "max_length":180}
)

after creating thr model we will create the chain

In [94]:

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [95]:

output_parser=StrOutputParser()

In [96]:

retriever=vector_db.as_retriever()

# vector_db = Weaviate.from_documents(
#     docs, embeddings, client=client, by_text=False
# )
# "context": retriever we have all the contex of docs , embeddign , client

In [97]:

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [98]:

print(rag_chain.invoke("what we called unit of charge"))

Human: You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: what we called unit of charge
Context: [Document(metadata={'page': 4, 'source': '/content/elctric_charges_book_RAG.pdf'}, page_content='was first suggested by the experimental laws of electrolysis discovered by\nEnglish experimentalist Faraday. It was experimentally demonstrated by\nMillikan in 1912.\nIn the International System (SI) of Units, a unit of charge is called a\ncoulomb  and is denoted by the symbol C. A coulomb is defined in terms\nthe unit of the electric current which you are going to learn in a\nsubsequent chapter . In ter ms of this definition, one coulomb is the char ge\nflowing through a wire in 1 s if the current is 1 A (ampere), (see Chapter 1\nof Class XI, Physics Textbook , Part I). In this system, the value of the\

The unit of charge is called a coulomb and is denoted by the symbol C. It is defined in terms of the unit of electric current, which is learned in a subsequent chapter. In the International System (SI) of Units, the value of the basic unit of charge is e = 1.602192 × 10–19 C. Thus, there are about 6 × 1018 electrons in a charge of –1

In [99]:
print(rag_chain.invoke("Explain the meaning of the statement ‘electric charge of a body is quantised."))

Human: You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: Explain the meaning of the statement ‘electric charge of a body is quantised.
Context: [Document(metadata={'page': 40, 'source': '/content/elctric_charges_book_RAG.pdf'}, page_content='quantity (e.g., angular momentum of an isolated system).\n8.Quantisation of electric charge is a basic (unexplained) law of nature;\ninterestingly, there is no analogous law on quantisation of mass.\n9.Superposition principle should not be regarded as ‘obvious’, or\nequated with the law of addition of vectors. It says two things:\nforce on one charge due to another charge is unaffected by the\npresence of other charges, and there are no additional three-body,\nfour-body, etc., for ces which arise only when ther e are mor e than\ntwo charges.\n10.The electr

"electric charge of a body is quantized" means that the total charge of a body is always an integral multiple of a basic quantum of charge, denoted by e. This basic unit of charge is the charge that an electron or proton carries. The fact that electric charge is always an integral multiple of e is termed as quantization of charge. This basic unit of charge is the same for all charged particles, regardless of their nature or size. The quantization of charge

In [100]:
print(rag_chain.invoke("""When a glass rod is rubbed with a silk cloth, charges appear on
both. A similar phenomenon is observed with many other pairs of
bodies. Explain how this observation is consistent with the law of
conservation of charge."""))

Human: You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: When a glass rod is rubbed with a silk cloth, charges appear on
both. A similar phenomenon is observed with many other pairs of
bodies. Explain how this observation is consistent with the law of
conservation of charge.
Context: [Document(metadata={'page': 1, 'source': '/content/elctric_charges_book_RAG.pdf'}, page_content='an entry which is called the electric char ge. We say that the bodies like\nglass or plastic rods, silk, fur and pith balls are electrified. They acquire\nan electric charge on rubbing. T here are two kinds of electrification and\nwe find  that (i) like charges repel  and (ii) unlike charges  attract  each\nother . The pr operty which dif ferentiates the two kinds of char ges is called\nthe polarity  of charge.\nWhen a

When a glass rod is rubbed with a silk cloth, charges appear on both the rod and the cloth. This phenomenon is consistent with the law of conservation of charge because the charges acquired by the glass rod and the silk cloth are opposite in polarity, and they repel each other. When the charged glass rod is brought in contact with the silk, the charges are lost, and the rod and the silk no longer attract each other. This is because the charges on the rod and the silk have